In [1]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [29]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [43]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
# mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	647	20_03_16_K5/
4	20-02-26	5	741	20_02_26_L3/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/


In [4]:
data = con.execute("""
SELECT NP, WAVELENGTH, DIP, DATE, COUNT, CON
FROM 'experiments' as EXP INNER JOIN 'measurements' as MEAS
   ON EXP.ID = MEAS.EXPERIMENT_ID;
   """)
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

80	740	727	20-02-26	128	3.7494866284721313
80	732	727	20-02-26	206	4.424635611736389
80	734	727	20-02-26	169	5.380528060883221
80	736	727	20-02-26	163	2.907049071433627
80	726	727	20-02-26	113	3.6359160741949874
80	724	727	20-02-26	137	3.053237526603112
60	732	727	20-02-26	166	3.968417288931015
60	732	727	20-02-26	234	3.816722879969379
80	651	647	20-03-16	54	3.8884484367508425
80	647	647	20-03-16	55	1.9014506031657823
80	649	647	20-03-16	66	2.9057549932215254


In [54]:
# 
columns = ' MASTER, WAVELENGTH, NP, DIP, FILE, RATE, CON, INP, AR_MIN, AR_COND, EXC_THS, EXP_TYPE,  NOTES, FOLDER'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
ORDER BY MASTER, CON DESC;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    


 MASTER	WAVELEN	NP	DIP	FILE	RATE	CON	INP	AR_MIN	AR_COND	EXC_THS	EXP_TYP	 NOTES	FOLDER
K4	653	100	647	raw_27_	11.54	4.87	9.90	6	1	[2.4]	-	-	20_03_1
K4	653	80	647	raw_06_	31.72	4.65	8.89	6	1	[2]	-	-	20_03_1
K4	651	80	647	raw_05_	29.07	3.89	10.07	6	2	[2.1]	-	-	20_03_1
K4	655	80	647	raw_07_	17.72	3.73	4.88	4	1	[1.5]	-	-	20_03_1
K4	653	60	647	raw_16_	27.27	3.33	5.11	6	1	[1.7]	-	-	20_03_1
K4	651	60	647	raw_13_	33.00	3.11	7.49	6	1	[2]	-	-	20_03_1
K4	649	80	647	raw_04_	26.32	2.91	5.14	6	2	[1.5]	-	-	20_03_1
K4	649	60	647	raw_14_	27.78	2.77	3.96	6	2	[1.7]	-	-	20_03_1
K4	651	100	647	raw_26_	19.10	2.70	6.45	6	2	[2.1]	-	bg fluc	20_03_1
K4	660	80	647	raw_08_	25.53	2.68	3.00	1	1	[1.4]	-	badly r	20_03_1
K4	647	80	647	raw_03_	15.56	1.90	3.11	6	1	[1.1]	-	-	20_03_1
K5	668	80	658	raw_06_	26.25	6.47	22.46	6	1	[2.5]	-	bg inte	20_03_1
K5	666	80	658	raw_05_	42.59	6.04	17.09	6	1	[2]	-	bg inte	20_03_1
K5	670	80	658	raw_07_	39.07	5.94	21.37	6	1	[2.5]	-	bg inte	20_03_1
K5	656	80	658	raw_09_	36.97	5.93	11.65	6	1	[

In [40]:
columns = 'NP, WAVELENGTH, X'
data = con.execute("""
SELECT {} FROM 'measurements';
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	X
80	740	16.71
80	732	32.05
80	734	24.95
80	736	23.03
80	726	41.31
80	724	40.87
60	732	27.16
60	732	28.07


In [53]:
con.execute("""
UPDATE 'measurements' SET NOTES = '-' WHERE WAVELENGTH = 656 OR WAVELENGTH = 654
""")

In [55]:
meas.commit()

In [9]:
con.close()